# Test Julynter

This is a simple notebook that uses selenium to check whether all lints appear in an external notebook. I made it because of the effort of manually checking all messages, but I didn't do as proper tests to avoid the effort of detecting elements appearing. As a notebook, I can run at the right pace.

Before running this notebook, there are some required steps:

1- Create a folder and move into the folder: `mkdir temp; cd temp`

2- Run jupyter lab externally (preferably in a clean environment with a fresh installation of Julynter): `jupyter lab`

3- Check the URL and replace the `URL` variable below to use the proper token

Now you can run this notebook cell by cell

In [1]:
from contextlib import contextmanager
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

URL = 'http://localhost:8888/?token=df789e7f5f4b2e73468ae4345afc18de69c8e3335defa481'

In [2]:
@contextmanager
def act(driver):
    actions = ActionChains(driver)
    try:
        yield actions
    finally:
        actions.perform()

In [3]:
def find_lints(driver, text):
    julynter_list = driver.find_element_by_css_selector('.jp-Julynter-content')
    return julynter_list.find_elements_by_xpath(
        '//div[contains(@class, "julynter-report-prompt")]/div[contains(., {!r})]'.format(text))

In [4]:
def reload_lints(driver):
    driver.find_element_by_css_selector('.julynter-title').click()

In [5]:
def alt_enter(actions):
    actions.key_down(Keys.ALT).send_keys(Keys.ENTER).key_up(Keys.ALT)

In [6]:
def shift_enter(actions):
    actions.key_down(Keys.SHIFT).send_keys(Keys.ENTER).key_up(Keys.SHIFT)

In [7]:
def write(driver, text, execute=True, before=""):
    with act(driver) as actions:
        if before:
            actions.send_keys(before)
        actions.send_keys(text)
        if execute:
            alt_enter(actions)


In [8]:
driver = webdriver.Firefox()

In [9]:
driver.get(URL)

In [10]:
side_bar = driver.find_element_by_css_selector('.jp-SideBar')
launcher = driver.find_element_by_css_selector('[title="New Launcher"]')
if not launcher.is_displayed():
    side_bar.find_element_by_css_selector('[title="File Browser (Ctrl+Shift+F)"]').click()

In [11]:
driver.find_element_by_css_selector('[title="New Launcher"]').click()

In [12]:
driver.find_element_by_css_selector('.p-DockPanel-widget:not(.lm-mod-hidden) [title="Python 3"]').click()

In [13]:
side_bar.find_element_by_css_selector('[title="Julynter"]').click()

In [14]:
# T2
title_lints = find_lints(driver, 'Title starts with "Untitled"')
assert len(title_lints) > 0
title_lints[0].click()

In [15]:
with act(driver) as actions:
    actions.send_keys(Keys.BACKSPACE + Keys.ENTER)
reload_lints(driver)

In [16]:
reload_lints(driver)

In [17]:
# T1
assert len(find_lints(driver, 'Title is empty')) > 0

In [18]:
# T7
title_lints = find_lints(driver, 'Title is too small')
assert len(title_lints) > 0
title_lints[0].click()

In [19]:
with act(driver) as actions:
    actions.send_keys("ã single title to rule them all but bigger better and stronger and even bigger so i dont know how big is too big-Copy.ipynb" + Keys.ENTER)


In [20]:
reload_lints(driver)

In [21]:
# T3, T4, T5, T6
assert len(find_lints(driver, 'Title has "-Copy"')) > 0
assert len(find_lints(driver, 'Title has blank spaces')) > 0
assert len(find_lints(driver, 'Title has special characters')) > 0
assert len(find_lints(driver, 'Title is too big')) > 0
find_lints(driver, 'Title is too big')[0].click()

In [22]:
with act(driver) as actions:
    actions.send_keys("NormalTitle" + Keys.ENTER)


In [23]:
reload_lints(driver)

In [24]:
# c4
lints = find_lints(driver, 'The first cell of the notebook is not a markdown cell')
assert len(lints) > 0
lints[0].click()

In [25]:
with act(driver) as actions:
    actions.send_keys(Keys.ESCAPE + "m" + Keys.ENTER)
reload_lints(driver)

In [26]:
driver.find_element_by_css_selector(':not(.lm-mod-hidden) .jp-Notebook').click()

In [27]:
write(driver, "title", before=Keys.ENTER)

In [28]:
write(driver, "if False:\nunknown")

In [29]:
# h6
assert len(find_lints(driver, "Cell 1 has the following undefined names: 'unknown'")) > 0

In [30]:
# c5
assert len(find_lints(driver, "The last cell of the notebook is not a markdown cell")) > 0

In [31]:
with act(driver) as actions:
    actions.send_keys(Keys.ESCAPE + "m" + Keys.ENTER)
reload_lints(driver)

In [32]:
write(driver, "templast")

In [33]:
write(driver, "a = 1", execute=False)

In [34]:
with act(driver) as actions:
    actions.send_keys(Keys.ESCAPE + "b" + Keys.ENTER)
reload_lints(driver)

In [35]:
write(driver, "b = 1")

In [36]:
# c1
assert len(find_lints(driver, "Cell 3 is a non-executed cell among executed ones")) > 0

In [37]:
write(driver, "")

In [38]:
write(driver, "c = b")

In [39]:
# c3
assert len(find_lints(driver, "Cell 5 is empty in the middle of the notebook")) > 0

In [40]:
with act(driver) as actions:
    actions.send_keys(Keys.ESCAPE + Keys.UP + Keys.UP + Keys.UP + Keys.UP + Keys.ENTER)
    shift_enter(actions)
    shift_enter(actions)
    actions.send_keys("dd")
reload_lints(driver)

In [41]:
# c2
assert len(find_lints(driver, "Cell 5 has the execution count 3 in the wrong order")) > 0

In [42]:
# h5
assert len(find_lints(driver, "Cell 5 uses name 'b' that was defined on In[2], but it does not exist anymore")) > 0

In [43]:
# h4
assert len(find_lints(driver, "Cell 5 skips the execution count")) > 0

In [44]:
with act(driver) as actions:
    actions.send_keys(Keys.TAB)

In [45]:
driver.find_element_by_css_selector(':not(.lm-mod-hidden) .jp-Notebook').click()

In [46]:
with act(driver) as actions:
    actions.send_keys(Keys.ENTER)

In [47]:
with act(driver) as actions:
    actions.send_keys(" + 1" + Keys.ESCAPE + Keys.UP)


In [48]:
# h2
lints = find_lints(driver, "Cell 5 has changed since its execution, but it wasn't executed after the changes")
assert len(lints) > 0


In [49]:
with act(driver) as actions:
    actions.send_keys(Keys.DOWN)
    shift_enter(actions)
write(driver, "import julynter", before=Keys.ENTER)
reload_lints(driver)

In [50]:
# i1
assert len(find_lints(driver, "Cell 6 has imports but it is not the first cell")) > 0

In [51]:
# i2
assert len(find_lints(driver, "Module julynter was imported by Cell 6, but it is not in the requirements file")) > 0

In [52]:
driver.find_element_by_css_selector(':not(.lm-mod-hidden) .jp-Notebook').click()

In [53]:
write(driver, "temp = '/usr/bin'", before=Keys.ENTER)

In [54]:
reload_lints(driver)

In [55]:
# p1
assert len(find_lints(driver, "Cell 7 has the following absolute paths: '/usr/bin'")) > 0

In [56]:
driver.find_element_by_css_selector(':not(.lm-mod-hidden) .jp-Notebook').click()

In [57]:
with act(driver) as actions:
    actions.send_keys(Keys.ESCAPE + "00")

In [58]:
with act(driver) as actions:
    actions.send_keys(Keys.ENTER)
reload_lints(driver)

In [59]:
# h1
assert len(find_lints(driver, "Cell 1 has execution results, but it wasn't executed on this session")) > 0

In [60]:
driver.find_element_by_css_selector(':not(.lm-mod-hidden) .jp-Notebook').click()
with act(driver) as actions:
    actions.send_keys(Keys.ESCAPE + Keys.UP + Keys.UP)

In [61]:
with act(driver) as actions:
    shift_enter(actions)
reload_lints(driver)

In [62]:
# h3
assert len(find_lints(driver, "Cell 7 repeats the execution count 1")) > 0

In [63]:
driver.find_element_by_css_selector(':not(.lm-mod-hidden) .jp-Notebook').click()

In [64]:
# Cleanup
write(driver, "!rm NormalTitle.ipynb", before=Keys.ENTER)

In [65]:
with act(driver) as actions:
    actions.key_down(Keys.ALT).send_keys("W").key_up(Keys.ALT)

In [66]:
with act(driver) as actions:
    actions.send_keys(Keys.LEFT + Keys.ENTER)

# Finished!
Make sure to check that the notebook `NormalTitle.ipynb` were deleted